This notebook uses the [GPyTorch package](https://github.com/cornellius-gp/gpytorch) to apply Gaussian Process regression to the multi-step energy consumption forecasting problem.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gpytorch
import torch

from gpytorch.kernels import ScaleKernel, LinearKernel, PeriodicKernel, AdditiveKernel, ProductKernel
from gpytorch.priors import NormalPrior
from sklearn.preprocessing import MinMaxScaler

In [ ]:
random_seed = 1923

Using the torch.float32 datatype seems to cause numerical stability issues, especially with variational inference & zero values. Using float64 and disabling mixed precision fixes them but slows down training quite a bit.

In [ ]:
# Set Torch settings
torch.set_default_dtype(torch.float64)
#torch.set_float32_matmul_precision('medium')

In [ ]:
# Plot settings
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [ ]:
output_dir = "./OutputData/"

In [ ]:
df = pd.read_csv(output_dir + "full_data.csv")
df["time"] = pd.to_datetime(df["time"], format = "%d:%m:%Y:%H:%M")

In [ ]:
# Drop generation columns
gen_cols = df.columns.values[2:].tolist()
df = df.drop(gen_cols, axis = 1)

In [ ]:
df

## Data prep

We do not need to cyclical encode seasonal features, as we will apply periodic kernels to them.

In [ ]:
# Add time columns

# Trend
df["trend"] = df.index.values

# Hour of day
df["hour"] = df.time.dt.hour + 1

# Day of week
df["dayofweek"] = df.time.dt.dayofweek + 1

# Month
df["month"] = df.time.dt.month

In [ ]:
df

In [ ]:
# Split features & target
X = df.drop(["time", "consumption_MWh",], axis = 1).values
y = df["consumption_MWh"].values

In [ ]:
# Train-test split
first_t = df[df["time"] == '2022-10-18 16:00:00'].index[0] # First prediction point
X_train_raw, X_test_raw = X[:first_t, :], X[first_t:, :]
y_train_raw, y_test_raw = y[:first_t], y[first_t:]

In [ ]:
# Feature & target scaling (0-1), tensor conversion
scaler = MinMaxScaler()
X_train = torch.tensor(scaler.fit_transform(X_train_raw))
X_test = torch.tensor(scaler.transform(X_test_raw))

scaler_target = MinMaxScaler()
y_train = torch.tensor(scaler_target.fit_transform(y_train_raw.reshape(-1, 1))).squeeze(-1)
y_test = torch.tensor(scaler_target.transform(y_test_raw.reshape(-1, 1))).squeeze(-1)

In [ ]:
# Subset training data to fit into GPU memory
train_size = int(24 * 365 * 1)
X_train = X_train[-train_size:, :]
y_train = y_train[-train_size:]

In [ ]:
X_train.shape

## Model & wrapper definition

Summary of training strategies tried:
- ExactGP can only be trained with unbatched gradient descent. The full training data does not fit into GPU memory. Training with the last ~9k observations in the training set works quite well for predicting the first few days of the testing set. Predictions move away from the series level as the distance from the training set increases, which is expected. Could be solved by rolling online updates of training data.
- SVGP can be trained with with batched gradient descent. Only fits into GPU memory with few inducing points, because inducing points are unbatched, kind of defeating the purpose of using SGD. Does not fit & converge properly, likely due to reliance on inducing points.
- VNNGP supports batching the inducing points, essentially using the entire data as both training data & inducing points. The initial computing of the k-nearest neighbor structure (when the model is created, not trained) is slow, but can be made faster by installing the faiss package. The entire data can be used as inducing points, but training loss is inexplicably high compared to SVGP (200k to 2k in 15-20 epochs), and predictions fluctuate around zero seemingly randomly. Also, constant warnings for negative variances are raised. I don't know why exactly this training approach fails to learn while the exact method does quite well. My theory is that the nearest neighbor structure somehow does not work with ordered time series data.

Based on all this, we continue with the ExactGP method, subsetting the training data. All of the strategies above were tried with the same covariance kernel choices: A linear kernel applied to the time series index (trend dummy), and a separate periodic kernel applied to each seasonal feature (hour, day of week, month of year), all combined additively.
- Applying all kernels to just the time index failed to learn the seasonality properly.

In [ ]:
# ExactGP model class
class ExactGPModel(gpytorch.models.ExactGP):

    def __init__(self, X_train, y_train, likelihood):
        super().__init__(X_train, y_train, likelihood)

        # Create mean module
        self.mean_module = gpytorch.means.ConstantMean()

        # Create covariance module
        self.covar_module = AdditiveKernel(
            LinearKernel(active_dims = 0), # Linear trend
            ScaleKernel(PeriodicKernel(
                active_dims = (1),
                period_length_prior = NormalPrior(24, 1) # Hourly seasonality
            )),
            ScaleKernel(PeriodicKernel(
                active_dims = (2),
                period_length_prior = NormalPrior(7, 1) # Day of week seasonality
            )),
            ScaleKernel(PeriodicKernel(
                active_dims = (3),
                period_length_prior = NormalPrior(12, 1) # Year of month seasonality
            )),
        )

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

In [ ]:
# ExactGP wrapper class (unbatched gradient descent)
class ExactGP:
    
    def __init__(self, model, likelihood, cuda = True):
        self.cuda = cuda

        # Put model & likelihood on GPU if cuda is enabled
        if cuda:
            self.model = model.cuda()
            self.likelihood = likelihood.cuda()
        else:
            self.model = model
            self.likelihood = likelihood
            
    # Training method
    def train(self, X_train, y_train, max_epochs, learning_rate = 0.01, early_stop = 5, early_stop_tol = 1e-3):

        # Put data on GPU if cuda is enabled
        if self.cuda:
            X_train = X_train.cuda()
            y_train = y_train.cuda()

        # Put models into training mode
        self.model.train()
        self.likelihood.train()

        # Create Adam optimizer with model parameters
        optimizer = torch.optim.Adam(self.model.parameters(), lr = learning_rate)

        # Create marginal log likelihood loss
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)

        # Training loop
        for epoch in range(max_epochs):

            # Reset gradients
            optimizer.zero_grad()

            # Get outputs from model
            output = self.model(X_train)

            # Calculate loss and perform backpropagation
            loss = -mll(output, y_train)
            loss.backward()
            optimizer.step()

            # Get loss & noise values to be printed
            loss_scalar = loss.item()
            noise = self.model.likelihood.noise.item()
            
            # Initialize best loss & rounds with no improvement if first epoch
            if epoch == 0:
                self._best_epoch = epoch
                self._best_loss = loss_scalar
                self._epochs_no_improvement = 0
                self._best_state_dict = self.model.state_dict()

            # Record an epoch with no improvement
            if self._best_loss < loss_scalar - early_stop_tol:
                self._epochs_no_improvement += 1

            # Record an improvement in the loss
            if self._best_loss > loss_scalar + early_stop_tol:
                self.best_epoch = epoch
                self._best_loss = loss_scalar
                self._epochs_no_improvement = 0
                self._best_state_dict = self.model.state_dict()

            # Print epoch summary
            print(f"Epoch: {epoch+1}/{max_epochs}, Loss: {loss_scalar}, Noise: {noise}, Best loss: {self._best_loss}")

            # Early stop if necessary
            if self._epochs_no_improvement >= early_stop:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Load best checkpoint after training 
        self.model.load_state_dict(self._best_state_dict)

        # Delete unneeded tensors
        del X_train, y_train, optimizer, mll, output, loss, loss_scalar, noise

        # Clear GPU memory 
        torch.cuda.empty_cache()
            
    # Predict method
    def predict(self, X_test, cpu = True, fast_preds = False):

        # Test data to GPU, if cuda enabled
        if self.cuda:
            X_test = X_test.cuda()

        # Activate eval mode
        self.model.eval()
        self.likelihood.eval()

        # Make predictions without gradient calculation
        with torch.no_grad(), gpytorch.settings.fast_pred_var(state = fast_preds):

            # Returns the model posterior distribution over functions: p(f*|X_test, X_train, y_train)
            # Noise is not yet added to the functions
            f_posterior = self.model(X_test)

            # Returns the predictive posterior distribution: p(y*|X_test, X_train, y_train)
            # Noise is added to the functions
            y_posterior = self.likelihood(f_posterior)

            # Get posterior predictive mean & prediction intervals
            # By default, 2 standard deviations around the mean
            y_mean = y_posterior.mean
            y_lower, y_upper = y_posterior.confidence_region()

        # Return predictions to CPU if desired
        # Could be advantageous to stack the predictions into one array
        if cpu:
            y_mean = y_mean.cpu()
            y_lower = y_lower.cpu()
            y_upper = y_upper.cpu()

        # Delete unneeded tensors
        del X_test, f_posterior, y_posterior

        # Clear GPU memory
        torch.cuda.empty_cache()

        return y_mean, y_lower, y_upper

    # Method to update model training data (kernel hyperparameters unchanged, no additional training performed)
    # When done repeatedly, GPU memory fills up, couldn't determine exact cause or find a fix.
    def update_train(self, X_update, y_update):
        
        # Put tensors on GPU if cuda is enabled
        if self.cuda:
            X_update = X_update.cuda()
            y_update = y_update.cuda()

        # Update model training data
        self.model = self.model.get_fantasy_model(X_update, y_update)

        # Delete unneeded tensors
        del X_update, y_update

        # Clear GPU memory
        torch.cuda.empty_cache()

    # Method to save model state dict
    def save(self, save_dir):
        torch.save(self.model.state_dict(), save_dir)

    # Method to load model parameters from saved state dict
    def load(self, state_dict):
        self.model.load_state_dict(state_dict)

## Model training & testing without online updates

In [ ]:
# Create likelihood, model, wrapper
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train, y_train, likelihood)
trainer = ExactGP(model, likelihood)

In [ ]:
# Load saved model state
model_name = "ExactGP1.pth"
state_dict = torch.load(output_dir + model_name)
trainer.load(state_dict)

Inference is also GPU intensive with a large dataset, so we subset the testing data to the training data size.
\
Wouldn't be a problem in model deployment, as we would keep the forecast horizon short, and load the trained model parameters from a .pth file without performing training in the same run.

In [ ]:
# Get predictions
preds_mean, preds_upper, preds_lower = trainer.predict(X_test[:train_size, :])

In [ ]:
# Backtransform predictions to original scale
preds_mean = scaler_target.inverse_transform(preds_mean.numpy().reshape(-1, 1)).squeeze()
preds_upper = scaler_target.inverse_transform(preds_upper.numpy().reshape(-1, 1)).squeeze()
preds_lower = scaler_target.inverse_transform(preds_lower.numpy().reshape(-1, 1)).squeeze()

In [ ]:
# Plot predicted vs. actual, first N days of testing data
plot_hours = 24 * 5

_ = sns.lineplot(
    x = X_test[:plot_hours, 0], 
    y = y_test_raw[:plot_hours], 
    label = "Actual")

_ = sns.lineplot(
    x = X_test[:plot_hours, 0], 
    y = preds_mean[:plot_hours], 
    label = "Predicted, 2sd interval")

_ = plt.fill_between(
    X_test[:plot_hours, 0], 
    preds_lower[:plot_hours], 
    preds_upper[:plot_hours], 
    color = "orange", 
    alpha = 0.4)

_ = plt.title(f"Exact GP predictions, no online training data updates\nForecast horizon: {plot_hours} hours\nTraining data length: {len(y_train)} hours")
_ = plt.xlabel("Scaled time index, 1 = end of train set")
_ = plt.ylabel("Hourly energy consumption")

In [ ]:
# Plot predicted vs. actual, entire test set
_ = sns.lineplot(
    x = X_test[:train_size, 0], 
    y = y_test_raw[:train_size], 
    label = "Actual")

_ = sns.lineplot(
    x = X_test[:train_size, 0], 
    y = preds_mean, 
    label = "Predicted, 2sd interval")

_ = plt.fill_between(
    X_test[:train_size, 0], 
    preds_lower, 
    preds_upper,
    color = "orange",
    alpha = 0.4)

_ = plt.title(f"Exact GP predictions, no online training data updates\nForecast horizon: {train_size} hours\nTraining data length: {len(y_train)} hours")
_ = plt.xlabel("Scaled time index, 1 = end of train set")
_ = plt.ylabel("Hourly energy consumption")

The initial predictions for the first few days are pretty good, and the kernels seem to be suitable for learning the seasonality of the data.
\
\
As the time index moves away from the last training point, the forecasts considerably deviate from the level of the data, but that is very natural with an effective forecast horizon of ~9k points. The lack of a past target covariate may be preventing the model from reacting to short-term cyclicality, as the LSTM and Transformer did not have this problem overall. 

## Model testing with online updates

It's possible to update the GP's training data with new data, extending its relevance to longer forecast horizons. Hyperparameters are not updated, no additional training or optimization takes place. This mirrors the usage of input sequences during prediction time in time series forecasting NN models. We condition the model on new data, essentially providing it with new context.
\
\
After each prediction, we will extend the training data in rolling fashion, with the newly predicted datapoints and the following 24 hours. This way, each new prediction starts from 16:00. 
\
\
Issue: GPU memory fills up as the model training data is updated. The loop still returns the predictions made before the GPU runs out of memory, so no need to rerun with a subset of the data. Again, this wouldn't be a problem in deployment, as we would likely do a single model update + prediction in one run, with a short horizon. We can also retrain the model with training data shifted forward in time, keeping the training data size constant as we extend it into the future.

In [ ]:
# Evaluation parameters that match the sequence2sequence testing scheme
horizon = 32 # Forecast horizon
stride = 24 # Number of timesteps between each prediction point
first_t = df[df["time"] == '2022-10-18 16:00:00'].index[0] # First prediction point
last_t = df.loc[(df.time.dt.hour == 16) & (df.index + horizon - 1 <= df.index.values[-1])].index.values[-2] # Last prediction point
n_windows = len(range(first_t + 1, last_t + 1, stride)) # N. of eval windows after first

In [ ]:
# Sequence2sequence testing loop

# Initialize predictions with first forecast horizon
preds_mean, preds_upper, preds_lower = trainer.predict(X_test[:horizon, :])

# Initialize predictions' time step indices
preds_indices = X_test[:horizon, 0]

# Update model training points with predicted points & steps before next prediction point
train_end = horizon + stride
trainer.update_train(
    X_test[:train_end, :],
    y_test[:train_end]
)

# Repeat for remaining evaluation windows
for window in range(0, n_windows):

    # Make predictions
    mean, upper, lower = trainer.predict(
        X_test[train_end:(train_end + horizon), :]
    )

    # Get predictions' time step indices
    indices = X_test[train_end:(train_end + horizon), 0]

    # Concatenate predictions & indices
    preds_mean = torch.cat((preds_mean, mean))
    preds_upper = torch.cat((preds_upper, upper))
    preds_lower = torch.cat((preds_lower, lower))
    preds_indices = torch.cat((preds_indices, indices))

    # Update model training points with predicted points & steps before next prediction point
    new_train_end = train_end + horizon + stride
    trainer.update_train(
        X_test[train_end:new_train_end, :],
        y_test[train_end:new_train_end]
    )
    train_end = new_train_end

    # Delete unused tensors & clear GPU memory
    del mean, upper, lower, indices
    torch.cuda.empty_cache()

In [ ]:
preds_mean.shape

In [ ]:
preds_indices.shape

We're able to predict roughly ~2k out of the previously predicted ~9k testing observations.

In [ ]:
# Backtransform predictions to original scale
preds_mean = scaler_target.inverse_transform(preds_mean.numpy().reshape(-1, 1)).squeeze()
preds_upper = scaler_target.inverse_transform(preds_upper.numpy().reshape(-1, 1)).squeeze()
preds_lower = scaler_target.inverse_transform(preds_lower.numpy().reshape(-1, 1)).squeeze()

In [ ]:
# Plot predicted vs. actual, entire test set
last_pred_idx = np.where(X_test[:, 0].numpy() == preds_indices[-1].numpy())[0][0]

_ = sns.lineplot(
    x = X_test[:last_pred_idx, 0], 
    y = y_test_raw[:last_pred_idx], 
    label = "Actual")

_ = sns.lineplot(
    x = preds_indices, 
    y = preds_mean, 
    label = "Predicted, 2sd interval")

_ = plt.fill_between(
    preds_indices, 
    preds_lower, 
    preds_upper, 
    color = "orange", 
    alpha = 0.4)

_ = plt.title(f"Exact GP predictions with online training data updates\nForecast horizon: {horizon} hours\nTraining data length: {len(y_train)} hours")
_ = plt.xlabel("Scaled time index, 1 = end of train set")
_ = plt.ylabel("Hourly energy consumption")

The plot is a little bit misleading, as the predicted values line (orange) is only plotted for the x-axis values where a prediction was made. The gaps, which correspond to the 24-hour stride periods without predictions, are connected with straight lines. Still, we can see the forecasts now continue to match the level of the data quite well as the predictions extend into the future. The forecast intervals also cover the actual values pretty well, which could be all one can ask for.

To sum up, the GP forecasting approach is probably best considered when:
- The training time series is not too long, or the near history is sufficient to make good predictions,
- The number of covariates is few, and the kernel choices for them are straightforward,
- Seasonality is strong & fixed (or there are other strongly predictive & easily available future covariates),
- Either a fixed trend is present, or the series level is constant,
- Cyclicality is low, or to put it another way, the additional predictive value of the past target values (or additional past covariates) are low,
- Online training data updates can be used to extend the model's relevance into the future,
- The predictions are not required real-time.